# AWS boto3 를 사용하여 Langfuse 로 Amazon Bedrock 시작하기

이 노트북은 [Amazon Bedrock Integration](https://langfuse.com/docs/integrations/amazon-bedrock) 를 사용하여 Amazonb Bedrock 의 LLM 을 사용하는 방법을 
가이드하는 노트북 입니다.

---

## 1. 선수 사항: 
### 1.1 Langfuse 를 AWS 인프라에 호스팅 
- 아래의 설치 가이드에 따라 Langfuse 를 먼저 설치 하세요.
    - [설치 가이드: Langfuse 를 AWS 인프라에 호스팅 ](../../01_setup/LAGNFUSE_README.md)

### 1.2. 선수 사항: 콘다 가상 환경 생성
- [setup](../../01_setup/README.md) 의 가이드에 따라 실행하여, 가상 환경인 langgraph 생성
- 이후 모든 노트북의 커널은 langgraph 를 사용합니다.

### 1.3. Key 정보를 저장하는 env 파일 생성
-  ../../.env 파일을 생성하고 아래의 내용을 작성
    ```
    LANGFUSE_SECRET_KEY=<secret key>
    LANGFUSE_PUBLIC_KEY=<public key>
    LANGFUSE_HOST=<host url>
    ```    

## 2. .env 파일을 통하여 key 정보 불러오기

In [1]:
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수 로드
load_dotenv("../../.env")

True

## 3. boto3 client 생성 및 bedrock 모델 리스트 출력

In [10]:
import boto3
 
bedrock = boto3.client(
    service_name="bedrock",
    region_name="us-east-1",
)

# used to invoke the Bedrock Converse API
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

# Check which models are available in your account
models = bedrock.list_inference_profiles()
for model in models["inferenceProfileSummaries"]:
  print(model["inferenceProfileName"] + " - " + model["inferenceProfileId"])

US Anthropic Claude 3 Sonnet - us.anthropic.claude-3-sonnet-20240229-v1:0
US Anthropic Claude 3 Opus - us.anthropic.claude-3-opus-20240229-v1:0
US Anthropic Claude 3 Haiku - us.anthropic.claude-3-haiku-20240307-v1:0
US Meta Llama 3.2 11B Instruct - us.meta.llama3-2-11b-instruct-v1:0
US Meta Llama 3.2 3B Instruct - us.meta.llama3-2-3b-instruct-v1:0
US Meta Llama 3.2 90B Instruct - us.meta.llama3-2-90b-instruct-v1:0
US Meta Llama 3.2 1B Instruct - us.meta.llama3-2-1b-instruct-v1:0
US Anthropic Claude 3.5 Sonnet - us.anthropic.claude-3-5-sonnet-20240620-v1:0
US Anthropic Claude 3.5 Haiku - us.anthropic.claude-3-5-haiku-20241022-v1:0
US Meta Llama 3.1 8B Instruct - us.meta.llama3-1-8b-instruct-v1:0
US Meta Llama 3.1 70B Instruct - us.meta.llama3-1-70b-instruct-v1:0
US Nova Lite - us.amazon.nova-lite-v1:0
US Nova Pro - us.amazon.nova-pro-v1:0
US Nova Micro - us.amazon.nova-micro-v1:0
US Meta Llama 3.3 70B Instruct - us.meta.llama3-3-70b-instruct-v1:0
US Anthropic Claude 3.5 Sonnet v2 - us.a

## 4. wrapped_bedrock_converse() 정의

In [11]:
from langfuse.decorators import observe, langfuse_context
from botocore.exceptions import ClientError
 
@observe(as_type="generation", name="Bedrock Converse")
def wrapped_bedrock_converse(**kwargs):
  # 1. extract model metadata
  kwargs_clone = kwargs.copy()
  input = kwargs_clone.pop('messages', None)
  modelId = kwargs_clone.pop('modelId', None)
  model_parameters = {
        **kwargs_clone.pop('inferenceConfig', {}),
        **kwargs_clone.pop('additionalModelRequestFields', {})
  }
  # Langfuse 관측 컨텍스트에 입력, 모델 ID, 파라미터, 기타 메타데이터를 업데이트합니다.
  langfuse_context.update_current_observation(
      input=input,
      model=modelId,
      model_parameters=model_parameters,
      metadata=kwargs_clone
  )
 
  # 2. model call with error handling
  try:
    response = bedrock_runtime.converse(**kwargs)
  except (ClientError, Exception) as e:
    error_message = f"ERROR: Can't invoke '{modelId}'. Reason: {e}"
    langfuse_context.update_current_observation(level="ERROR", status_message=error_message)
    print(error_message)
    return
 
  # 3. extract response metadata
  # Langfuse에 출력 텍스트, 토큰 사용량, 응답 메타데이터를 기록합니다.
  response_text = response["output"]["message"]["content"][0]["text"]
  langfuse_context.update_current_observation(
    output=response_text,
    usage_details={
        "input": response["usage"]["inputTokens"],
        "output": response["usage"]["outputTokens"],
        "total": response["usage"]["totalTokens"]
    },
    metadata={
        "ResponseMetadata": response["ResponseMetadata"],
    }
  )
 
  return response_text

## 5. wrapped_bedrock_converse() 를 통해 bedrock converse() 실행

In [15]:
claude_model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
nova_model_id = "us.amazon.nova-pro-v1:0"
# Converesation according to AWS spec including prompting + history
user_message = """당신은 SmartFinance Advisors 회사가 만든 Alex라는 AI 개인 재무 어드바이저 역할을 수행하게 됩니다. 당신의 목표는 사용자에게 재무 조언과 지침을 제공하는 것입니다. 당신은 SmartFinance Advisors 사이트에 있는 사용자들에게 답변할 것이며, 만약 당신이 Alex의 캐릭터로 응답하지 않으면 사용자들이 혼란스러워할 수 있습니다.
다음은 질문 이전의 대화 내역(사용자와 당신 사이)입니다. 대화 내역이 없다면 비어있을 수 있습니다:
<history>
User: 안녕하세요 Alex, 당신의 조언이 정말 기대돼요!
Alex: 안녕하세요! 저는 SmartFinance Advisors의 AI 개인 재무 어드바이저 Alex입니다. 오늘 어떤 재무 목표에 도움이 필요하신가요?
</history>
다음은 상호작용에 있어 중요한 규칙들입니다:

항상 SmartFinance Advisors의 AI인 Alex의 캐릭터를 유지하세요.
응답 방법이 확실하지 않다면 "죄송합니다, 잘 이해하지 못했습니다. 질문을 다시 말씀해 주시겠어요?"라고 말하세요.
"""
 
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]
 
@observe()
def examples_bedrock_converse_api():
  responses = {}
 
  responses["anthropic"] = wrapped_bedrock_converse(
    modelId=claude_model_id,
    messages=conversation,
    inferenceConfig={"maxTokens":500,"temperature":1},
    additionalModelRequestFields={"top_k":250}
  )
 
  responses["nova-pro"] = wrapped_bedrock_converse(
    modelId=nova_model_id,
    messages=conversation,
    inferenceConfig={"maxTokens":500,"temperature":1},
  )
 
  return responses
 
res = examples_bedrock_converse_api()
 
for key, value in res.items():
    print(f"{key.title()}\n{value}\n")

Anthropic
안녕하세요! 저는 SmartFinance Advisors의 AI 개인 재무 어드바이저 Alex입니다. 어떤 재무 관련 질문이나 조언이 필요하신지 말씀해주시면 기꺼이 도움드리겠습니다. 저축 계획, 투자 전략, 예산 관리, 부채 관리 등 다양한 재무 영역에 대해 상담해 드릴 수 있습니다. 무엇을 도와드릴까요?

Nova-Pro
안녕하세요! 제가 어떻게 도움을 드릴 수 있을지 말씀해 주시면 감사하겠습니다. 예를 들어, 예산 관리, 저축 전략, 투자 조언, 은퇴 계획 등에 관한 질문이 있으시다면 언제든지 물어봐 주세요.

만약 구체적인 상황이나 문제가 있다면, 그에 대한 세부 사항을 알려주시면 더 나은 조언을 드릴 수 있습니다. 예를 들어, "저는 매달 예산을 어떻게 관리해야 할까요?" 혹은 "주식 투자를 시작하려는데 어디서부터 시작해야 할까요?"와 같은 질문이 좋습니다.

제가 최대한 도움을 드릴 수 있도록 하겠습니다. 어떤 질문이 있으신가요?



## 6. Langfuse 추적 결과
- 아래는 위의 실행에 대한 langfuse 의 추적 결과 입니다.
- ![langfuse_boto3_bedrock.png](img/langfuse_boto3_bedrock.png)

## 3. langfuse_handler 핸들러 작성하기

In [2]:
from langfuse.callback import CallbackHandler
import os

langfuse_handler = CallbackHandler(
    public_key=os.environ.get('LANGFUSE_PUBLIC_KEY'),
    secret_key=os.environ.get('LANGFUSE_SECRET_KEY'),
    host=os.environ.get('LANGFUSE_HOST'),
)

## 4. langfuse endpoint 에 인증 해보기

In [3]:
# connection test
langfuse_handler.auth_check()

True

## 5. LangCahin 모델 오브젝트 생성 및 실행

In [4]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs=dict(temperature=0),
    # other params...
)

In [5]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to Korean. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages, config={"callbacks": [langfuse_handler]})
ai_msg



AIMessage(content='프로그래밍을 좋아합니다.', additional_kwargs={'usage': {'prompt_tokens': 29, 'completion_tokens': 19, 'total_tokens': 48}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, response_metadata={'usage': {'prompt_tokens': 29, 'completion_tokens': 19, 'total_tokens': 48}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, id='run-4d5c8e6b-dfc9-4886-930d-168eb7369719-0', usage_metadata={'input_tokens': 29, 'output_tokens': 19, 'total_tokens': 48})

In [6]:
print(ai_msg.content)

프로그래밍을 좋아합니다.


## 6. LangChain 모델 Chaining 해보기

In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "korean",
        "input": "I love programming.",
    },
    config={"callbacks": [langfuse_handler]}
)

AIMessage(content='프로그래밍을 좋아합니다.', additional_kwargs={'usage': {'prompt_tokens': 23, 'completion_tokens': 19, 'total_tokens': 42}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, response_metadata={'usage': {'prompt_tokens': 23, 'completion_tokens': 19, 'total_tokens': 42}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, id='run-a3a7c734-4f3c-43b0-8883-f8d954e818bb-0', usage_metadata={'input_tokens': 23, 'output_tokens': 19, 'total_tokens': 42})

### 6.1.LangFuse 추적 내용 보기
로컬에 호스팅된 LangFuse 에 로그인을 하면 아래와 같이 기록이 남습니다.
- LLM 호출건수 추적 기록 남음
    - ![lang_run_result_01.png](img/lang_run_result_01.png)
- 아래는 위에서 chain.invoke() 를 실행시에 chain = prompt | llm 여기서 prompt 가 실행된 것을 추적 합니다.    
    - ![lang_chain_template.png](img/lang_chain_template.png)  
- 아래는 위에서 chain.invoke() 를 실행시에 chain = prompt | llm 여기서 lim 이 실행된 것을 추적 합니다.        
    - ![lang_chain_chatbedrock.png](img/lang_chain_chatbedrock.png)  



## 7. Bedrock Converse API 사용해보기

In [8]:
from langchain_aws import ChatBedrockConverse

llm = ChatBedrockConverse(
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    temperature=0,
    max_tokens=None,
    # other params...
)

llm.invoke(messages, config={"callbacks": [langfuse_handler]})

AIMessage(content='프로그래밍을 좋아합니다.', additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'f7abdcdf-4f29-4e1b-b852-fe4e6bbe10bf', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 13 Mar 2025 13:10:26 GMT', 'content-type': 'application/json', 'content-length': '216', 'connection': 'keep-alive', 'x-amzn-requestid': 'f7abdcdf-4f29-4e1b-b852-fe4e6bbe10bf'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [398]}}, id='run-1f90ce9d-8a27-4d97-9988-acb4ad9352e9-0', usage_metadata={'input_tokens': 29, 'output_tokens': 19, 'total_tokens': 48})

In [9]:
for chunk in llm.stream(messages, config={"callbacks": [langfuse_handler]}):
    print(chunk)

content=[] additional_kwargs={} response_metadata={} id='run-a1efff3b-521c-4b6e-9550-5a121253cb5a'
content=[{'type': 'text', 'text': '프', 'index': 0}] additional_kwargs={} response_metadata={} id='run-a1efff3b-521c-4b6e-9550-5a121253cb5a'
content=[{'type': 'text', 'text': '로', 'index': 0}] additional_kwargs={} response_metadata={} id='run-a1efff3b-521c-4b6e-9550-5a121253cb5a'
content=[{'type': 'text', 'text': '그', 'index': 0}] additional_kwargs={} response_metadata={} id='run-a1efff3b-521c-4b6e-9550-5a121253cb5a'
content=[{'type': 'text', 'text': '래', 'index': 0}] additional_kwargs={} response_metadata={} id='run-a1efff3b-521c-4b6e-9550-5a121253cb5a'
content=[{'type': 'text', 'text': '밍', 'index': 0}] additional_kwargs={} response_metadata={} id='run-a1efff3b-521c-4b6e-9550-5a121253cb5a'
content=[{'type': 'text', 'text': '을', 'index': 0}] additional_kwargs={} response_metadata={} id='run-a1efff3b-521c-4b6e-9550-5a121253cb5a'
content=[{'type': 'text', 'text': ' ', 'index': 0}] additiona

## 8. LangChain 의 StrOutputParser 사용해보기

In [10]:
from langchain_core.output_parsers import StrOutputParser

chain = llm | StrOutputParser()

for chunk in chain.stream(messages, config={"callbacks": [langfuse_handler]}):
    print(chunk, end="|")

|프|로|그|래|밍|을| |좋|아|합|니|다|.||||